In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib agg

In [ ]:
import os, sys, re
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "lib/python"))

import numpy as np

# disable GUI
import matplotlib as mpl
mpl.use('Agg')

import matplotlib.animation as ani

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader

In [ ]:
def latex_float(number):
    float_str = "{0:.3g}".format(number)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0:.2f}\times10^{{{1}}}".format(float(base), int(exponent))
    else:
        return float_str

In [ ]:
# service options
delimiter = '/'

## configuration options
data_path = '/data/models/m40s1vXXXXXX/data_twobeams.h5'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, verbose=False)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

time_range = [reader.meta.time[0], reader.meta.time[1]]
frame_step = 1
# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
frame_size = [0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

cmap='terrain_r'
image_interpolation = 'nearest'

autoselect = True
show_grid=False
aspect='equal'

guess_number_ticks = 20

cmp_r = 'E/r'
cmp_z = 'E/z'
cmp_beam = 'density'
specie_beam = 'beam_electrons'


dry_run = False
view = False

video_file = "mesh_movie_3_E_beam_density_{}-{}.avi".format(time_range[0], time_range[1])

r_scale = reader.meta.geometry_size[0] / reader.meta.geometry_grid[0]
z_scale = reader.meta.geometry_size[1] / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
beam_density = reader.meta.beams[0].bunch_density
beam_density_delta = reader.meta.beams[0].bunch_density * 0.2

clim_r = [-clim_estimation, clim_estimation]
clim_z = [-clim_estimation, clim_estimation]
clim_beam = [0, beam_density + beam_density_delta]

x_axis_label = r'$\mathit{z\enspace (m)}$'
y_axis_label = r'$\mathit{r\enspace (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_beam_axis_label = r'$m^{-3}$'

plot_name_r = r'$\mathbf{Electric\enspace Field\enspace E_r\enspace Component}$'
plot_name_z = r'$\mathbf{Electric\enspace Field\enspace E_z\enspace Component}$'
plot_name_beam = r'$\mathbf{Electron\enspace Beam\enspace Density\enspace \rho_{beam}}$'

plot_name = "Electric Field Components ({}, {}) and Electron Beam Density".format(re.sub(delimiter, '_', cmp_r), re.sub(delimiter, '_', cmp_z))

In [ ]:
# define plot builder
plot = PlotBuilder(frame_size[3] - frame_size[1], frame_size[2] - frame_size[0],
                   fig_color=reader.meta.figure_color, 
                   fig_width=7,
                   fig_height=4.8, 
                   fig_dpi=300,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=8,
                   
                   x_ticklabel_start=frame_size[1] * z_scale,
                   y_ticklabel_start=frame_size[0] * r_scale,
                   x_ticklabel_end=frame_size[3] * z_scale,
                   y_ticklabel_end=frame_size[2] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect=aspect, image_interpolation=image_interpolation,
                   guess_number_ticks=guess_number_ticks)
o_left=0.100
o_right=0.890
o_top=0.970
o_bot=0.035

import matplotlib.pyplot as plt

plt.subplots_adjust(left=o_left,
                    bottom=o_bot,
                    right=o_right,
                    top=o_top,
                    wspace=0.010, 
                    hspace=0.010)


In [ ]:
# add subplots
subplot_r = plot.add_subplot_cartesian_2d(plot_name_r, 311, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
subplot_z = plot.add_subplot_cartesian_2d(plot_name_z, 312, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
subplot_b = plot.add_subplot_cartesian_2d(plot_name_beam, 313, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

subplot_r.yaxis.set_label_coords(-.09, .5)
subplot_r.xaxis.set_label_coords(.5, -.3)
subplot_z.yaxis.set_label_coords(-.09, .5)
subplot_z.xaxis.set_label_coords(.5, -.3)
subplot_b.yaxis.set_label_coords(-.09, .5)
subplot_b.xaxis.set_label_coords(.5, -.3)

# add initial image
initial_image = np.zeros([frame_size[2] - frame_size[0], frame_size[3] - frame_size[1]])

plot.add_image(plot_name_r, initial_image, cmap=cmap, clim=clim_r)
plot.add_image(plot_name_z, initial_image, cmap=cmap, clim=clim_z)
plot.add_image(plot_name_beam, initial_image, cmap=cmap, clim=clim_beam)

# add colorbars
plot.add_colorbar(plot_name_r, ticks=clim_r, title=cbar_axis_label)
plot.add_colorbar(plot_name_z, ticks=clim_z, title=cbar_axis_label)
plot.add_colorbar(plot_name_beam, ticks=clim_beam, title=cbar_beam_axis_label)

In [ ]:
schedule = 0

for p in reader.meta.probes:
    if p.component == cmp_r:
        schedule = p.schedule
        break
        
start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], schedule)
end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], schedule)

frame_src_size=[-1, -1, -1, -1]
    
# detect probe shape
for probe in reader.meta.probes:
    if ( probe.shape == 'rec'
        and probe.component == cmp_r
        and probe.size[0] == frame_size[0]
        and probe.size[1] == frame_size[1]
        and probe.size[2] == frame_size[2]
        and probe.size[3] == frame_size[3] ):
        frame_src_size = probe.size
        
    # try bigger frames, if autoselect enabled
    if ( ( frame_src_size[0] == -1 
          or frame_src_size[1] == -1 
          or frame_src_size[2] == -1 
          or frame_src_size[3] == -1 )
        and autoselect ):
        for probe in reader.meta.probes:
            if ( probe.shape == 'rec'
                and probe.component == cmp_r
                and probe.size[0] <= frame_size[0]
                and probe.size[1] <= frame_size[1]
                and probe.size[2] >= frame_size[2]
                and probe.size[3] >= frame_size[3] ):
                frame_src_size = probe.size

In [ ]:
FFMpegWriter = ani.writers['ffmpeg']

metadata = dict(title = plot_name)
writer = FFMpegWriter(fps = reader.meta.video_fps,
                      metadata = metadata,
                      codec = reader.meta.video_codec,
                      bitrate = reader.meta.video_bitrate)

if dry_run: video_file = '/dev/null'
fig = plot.get_figure()

with writer.saving(fig, video_file, 300):
    for i in range(start_frame, end_frame):
        if i % frame_step == 0:
            sys.stdout.write('Loading dataset ' + str(i) + '... ')
            sys.stdout.flush()
            data_r = reader.rec(cmp_r, frame_src_size, i)[frame_size[0]:frame_size[2], frame_size[1]:frame_size[3]]
            data_z = reader.rec(cmp_z, frame_src_size, i)[frame_size[0]:frame_size[2], frame_size[1]:frame_size[3]]
            data_beam = reader.rec("{}{}{}".format(cmp_beam, delimiter, specie_beam), frame_src_size, i)[frame_size[0]:frame_size[2], frame_size[1]:frame_size[3]]

# dirty hack. Something wrong with particle placement
            data_beam[128] = data_beam[127]
            data_beam[256] = data_beam[255]
            # data_beam[384] = data_beam[383]
            data_beam[:,128] = data_beam[:,127]
            data_beam[:,256] = data_beam[:,255]
            data_beam[:,384] = data_beam[:,383]
            data_beam[:,512] = data_beam[:,511]
            data_beam[:,640] = data_beam[:,639]
            data_beam[:,768] = data_beam[:,767]
            data_beam[:,896] = data_beam[:,895]
            data_beam[:,1024] = data_beam[:,1023]
            data_beam[:,1152] = data_beam[:,1151]
            data_beam[:,1280] = data_beam[:,1279]
            data_beam[:,1408] = data_beam[:,1407]
            data_beam[:,1536] = data_beam[:,1535]
            data_beam[:,1664] = data_beam[:,1663]
            data_beam[:,1792] = data_beam[:,1791]
            data_beam[:,1920] = data_beam[:,1919]

            # add timestamp to each frame
            timestamp = reader.meta.get_timestamp_by_frame_number(i, schedule)
            fig.suptitle('$\mathbf{t\enspace =\enspace ' + latex_float(timestamp) + 's}$', x=.80, y=.96)
            
            plot.add_image(plot_name_r, data_r, cmap=cmap, clim=clim_r)
            plot.add_image(plot_name_z, data_z, cmap=cmap, clim=clim_z)
            plot.add_image(plot_name_beam, data_beam, cmap=cmap, clim=clim_beam)

            if view: plot.redraw()
            if not dry_run: 
                writer.grab_frame()

            print('DONE')